In [1]:
from pathlib import Path
import os

yr = 2023
d = 6

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  for i, l in enumerate(inp.split('\n')):
    vals = [int(x) for x in l.split(' ') if x.isnumeric()]
    if i == 0:
      rtimes = vals
    elif i == 1:
      distances = vals
  return {rtimes[i]: distances[i] for i in range(len(rtimes))}

def format_kern_input(inp):
  for i, l in enumerate(inp.split('\n')):
    vals = [str(x) for x in l.split(' ') if x.isnumeric()]
    if i == 0:
      rtimes = vals
    elif i == 1:
      distances = vals
  return {int(''.join(rtimes)): int(''.join(distances))}

In [3]:
def get_distance(rtime, htime):
  mtime = rtime - htime
  return htime * mtime

def get_all_possible_distances(rtime):
  dists = []
  for htime in range(rtime+1):
    dists.append(get_distance(rtime, htime))
  return dists

def get_winning_distances(rtime, distance):
  possible_dists = get_all_possible_distances(rtime)
  return [d for d in possible_dists if d > distance]

def get_winning_distances_multi_race(formatted_input):
  output = []
  for t, d in formatted_input.items():
    output.append(get_winning_distances(t, d))
  return output

def get_winning_distances_multi_race_product(formatted_input):
  winners = get_winning_distances_multi_race(formatted_input)
  prod = 1
  for w in winners:
    prod = prod*len(w)
  return prod

In [4]:
def get_hold_times(rtime, d):
  from scipy.optimize import fsolve
  hts = fsolve(lambda x: d+1-get_distance(rtime, x), [0,rtime])
  return (int(hts[0]+1), int(hts[1]))


def get_possible_winning_hold_times(formatted_input_kern):
  rtime = list(formatted_input_kern.keys())[0]
  d = list(formatted_input_kern.values())[0]
  min_ht, max_ht = get_hold_times(rtime, d)
  return max_ht - min_ht + 1

In [5]:
import time

t = time.time()

formatted_input = format_input(inp)
formatted_input_kern = format_kern_input(inp)

print(get_winning_distances_multi_race_product(formatted_input))
print(get_possible_winning_hold_times(formatted_input_kern))

print("\nRUNTIME: ", time.time()-t)

633080
20048741

RUNTIME:  0.29919958114624023
